# 券商分點

這個券商分點可以繞過「圖片認證碼」請大家再三保密使用XD
不然哪天就失效了，

抓取券商分點，每檔股票總共可以分成三個步驟：
1. 開啟驗證碼網頁，得到驗證碼
2. 利用驗證碼來得到券商分點
3. 整理券商分點資訊

In [8]:
# ---------------
# 偽裝成瀏覽器
# ---------------
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

# 1. 開啟驗證碼網頁，得到驗證碼
這邊的驗證碼不是「圖片驗證碼喔！」而是 `viewstate` 跟 `eventvalidation`

In [9]:
# ---------------
# 開啟有驗證碼的網頁
# ---------------
import requests
rs = requests.session()

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

viewstate, eventvalidation = get_verify_code(rs)

print('----')
print('viewstate', viewstate)
print('----')
print('eventvalidation',eventvalidation)
print('----')

----
viewstate /wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkMDBmYTdiNDUtOWI4My00MmJjLTlmNjUtNjIzNzdiNTVhYmNkZCSsaSYebj1JdIpyT76ff9E=
----
eventvalidation /wEdAAacI4mpkrbQHJkRupXD+k1UfMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6GppMgxlY639prJ1UlGEmIPD
----


C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# 2. 透過驗證碼爬取資料並存檔

In [10]:
# ---------------
# 爬取資料
# ---------------

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }
    rs.post( "https://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers, stream = True )
    time.sleep(1)
    res = rs.get( 'https://bsr.twse.com.tw/bshtm/bsContent.aspx',verify = False ,stream = True, )
    return res

res = get_data(rs, '1101', viewstate, eventvalidation)
res.encoding='big5'
open('test.html', 'w', encoding='utf-8').write(res.text)

C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


36547

# 3. 整理資料

In [11]:
import pandas as pd
from io import StringIO

def parse_data(text):
    lines = text.split('\n')
    lines = [l for l in lines if len(l.split(',')) == 11]
    df = pd.read_csv(StringIO('\n'.join(lines)))

    first_df = df[df.columns[:5]]
    second_df = df[df.columns[6:]]
    second_df.columns = second_df.columns.str.replace('.1', '')
    final_df = first_df.append(second_df).set_index('序號').sort_index().dropna()
    return final_df




df = parse_data(res.text)

# 完整下載範例（每天來按一下）

假如無法下載，可以參考
http://finlabcourse.imotor.com/viewthread.php?tid=322&extra=page%3D1&page=3
來做一些更動，就可以順利執行了～

In [1]:
import time
import requests
from io import StringIO
import pandas as pd
import datetime

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "https://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'https://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

def get_verify_code(rs):
    res = rs.get( 'https://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

def download_stock(stock_id, path):
    rs = requests.Session()
    #viewstate, eventvalidation = get_verify_code(rs)
    
    while True:
        try:
            viewstate, eventvalidation = get_verify_code(rs)
            break # 抓資料成功,進行下一步
        except Exception as e:
            print(e) # 印出無法連線原因
            print('無法拿到資料, 等 31 sec') # server好像會擋30sec
            time.sleep(31)
            continue # 抓資料失敗,重新抓資料
    
    time.sleep(2)
    res = get_data(rs, stock_id, viewstate, eventvalidation)
    res.encoding='big5'
    open(path, 'w', encoding='utf-8').writelines(res.text)
    
def stock_list():
    res = requests.get('https://dts.twse.com.tw/opendata/t187ap03_L.csv')
    res.encoding='utf-8'
    df = pd.read_csv(StringIO(res.text), index_col=['公司代號'])
    return df

import os

# build main dir
main_dir = os.path.join('data', 'broker_history')
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
    
# build date dir
now = datetime.datetime.now()
date_dir = os.path.join(main_dir, now.strftime('%Y%m%d'))
if not os.path.isdir(date_dir):
    os.mkdir(date_dir)

    
slist = stock_list()
for s in slist.index:
    stock_path = os.path.join(date_dir, str(s) + '.csv')
    print(stock_path)
    if not os.path.isfile(stock_path):
        print('downloading...')
        download_stock(str(s), stock_path)
        time.sleep(10)
    

data\broker_history\20191228\1101.csv
downloading...


C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data\broker_history\20191228\1102.csv
downloading...


C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data\broker_history\20191228\1103.csv
downloading...


...

C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data\broker_history\20191228\9955.csv
downloading...


C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


data\broker_history\20191228\9958.csv
downloading...


C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\anaconda\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# 券商總買賣

In [12]:
buy = df['買進股數'].astype(int).groupby(df['券商']).sum()
sell= df['賣出股數'].astype(int).groupby(df['券商']).sum()
total = buy - sell
total

券商
1020合　　庫       34704
1021合庫台中      -13420
1022合庫台南           0
1023合庫高雄       -5909
1024合庫嘉義         300
1025合庫基隆        1000
1028合庫彰化       -9000
1029合庫鳳山        5000
102A合庫新竹       -1000
102C合庫自強       -8000
102E合庫桃園       -5070
102F合庫西台       -1000
102G合庫三重       -1070
1030土　　銀       -2000
1032土銀台南       -2000
1033土銀高雄       -3000
1034土銀嘉義       -6000
1035土銀新竹       -2000
1038土銀和平       -3000
103A土銀建國       -1000
103B土銀彰化        3000
103F土銀南港       -2000
1040臺    銀    -70350
1041臺銀鳳山       -2000
1042臺銀臺南       -1000
1043臺銀民權       13000
1045臺銀新竹       -5000
104A臺銀臺中      130860
104C臺銀高雄        9100
104D臺銀金山        4000
               ...  
9A97永豐新竹       18000
9A98永豐大園       -2000
9A9A永豐羅東      -20000
9A9B永豐中和       21100
9A9C永豐員林       -1000
9A9D永豐忠孝        6000
9A9G永豐天母       19930
9A9J永豐板新        6000
9A9K永豐三重        1000
9A9L永豐台中      -15000
9A9M永豐南投      -11000
9A9N永豐桃盛       -2630
9A9P永豐竹北       -1000
9A9R永豐信義        5000
9A9S永豐南京         250
9A9U永豐中正         893
9A9W永豐市政  

# 主力買賣超

In [13]:
# 買方top 15
total.nlargest(15)

券商
8440摩根大通      2234000
9200凱基        1110000
9872元大復北       875000
1440美林         554814
1470台灣摩根       538000
116r日盛文化       405000
9268凱基台北       165100
1560港商野村       131000
104A臺銀臺中       130860
5870光    隆     105860
8900法銀巴黎       100000
8560新    光      88799
9208凱基桃園        85879
1650瑞銀          75000
9272凱基竹科        70000
dtype: int32

In [14]:
# 賣方 top 15
total.nsmallest(15)

券商
5850統    一   -2027000
8711陽信石牌     -1302000
9100群益        -533000
1520瑞士信貸      -220184
1360港麥格理      -165000
1480美商高盛      -135261
9600富邦        -130700
980D元大開元      -103005
989d元大北屯      -102118
8565新光桃園       -95070
592D元富大昌       -90000
8848玉山嘉義       -85196
8150台    新     -76132
9234凱基竹北       -75007
1040臺    銀     -70350
dtype: int32

In [15]:
# 主力買賣超
total.nlargest(15).sum() + total.nsmallest(15).sum()

1459289

# 買賣家數差

In [16]:
# 買賣家數差
(total > 0).sum() - (total < 0).sum()

-250